In [1]:
import os
import requests
import json
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np
from datetime import datetime

In [2]:
coins = ['bitcoin', 'solana']

load_dotenv()

url_base = "https://api.coingecko.com/api/v3/coins/"
days = 1
params = {
    "vs_currency": "usd",
    "days": days
}

print(params)

{'vs_currency': 'usd', 'days': 1}


# EXTRACT

In [3]:
payloads = []


for idx, coin in enumerate(coins):
    url = f"{url_base}{coin}/ohlc"
    response = requests.get(url, params=params)
    payloads.append(response.json())
    
formatted_response = json.dumps(payloads, indent=4)


# TRASNFORM

In [4]:
ohlc_df1 = pd.DataFrame(payloads)
ohlc_df1

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,"[1758898800000, 109057.0, 109057.0, 108971.0, ...","[1758900600000, 109100.0, 109166.0, 109020.0, ...","[1758902400000, 109325.0, 109380.0, 109156.0, ...","[1758904200000, 109099.0, 109561.0, 109099.0, ...","[1758906000000, 109540.0, 109540.0, 109385.0, ...","[1758907800000, 109509.0, 109673.0, 109500.0, ...","[1758909600000, 109890.0, 109998.0, 109856.0, ...","[1758911400000, 109979.0, 110311.0, 109927.0, ...","[1758913200000, 110259.0, 110259.0, 109872.0, ...","[1758915000000, 109773.0, 109773.0, 109494.0, ...",...,"[1758967200000, 109159.0, 109347.0, 109159.0, ...","[1758969000000, 109330.0, 109334.0, 109319.0, ...","[1758970800000, 109334.0, 109466.0, 109334.0, ...","[1758972600000, 109477.0, 109477.0, 109345.0, ...","[1758974400000, 109359.0, 109377.0, 109334.0, ...","[1758976200000, 109343.0, 109343.0, 109295.0, ...","[1758978000000, 109302.0, 109437.0, 109283.0, ...","[1758979800000, 109432.0, 109432.0, 109352.0, ...","[1758981600000, 109396.0, 109396.0, 109362.0, ...","[1758983400000, 109351.0, 109382.0, 109351.0, ..."
1,"[1758898800000, 194.97, 194.97, 194.22, 194.22]","[1758900600000, 195.31, 196.06, 195.27, 196.06]","[1758902400000, 196.46, 196.61, 196.28, 196.28]","[1758904200000, 196.08, 198.92, 196.08, 198.92]","[1758906000000, 198.7, 199.56, 198.52, 199.49]","[1758907800000, 199.71, 200.16, 199.71, 200.16]","[1758909600000, 201.24, 201.24, 200.94, 201.05]","[1758911400000, 201.43, 202.43, 201.27, 202.43]","[1758913200000, 202.2, 202.28, 201.95, 201.95]","[1758915000000, 201.97, 201.97, 201.32, 201.43]",...,"[1758967200000, 200.29, 201.38, 200.29, 201.38]","[1758969000000, 201.28, 201.28, 200.8, 200.8]","[1758970800000, 200.83, 201.87, 200.83, 201.75]","[1758972600000, 201.71, 201.84, 201.59, 201.84]","[1758974400000, 201.93, 202.03, 201.6, 201.66]","[1758976200000, 201.69, 201.69, 201.22, 201.34]","[1758978000000, 201.32, 202.44, 201.32, 202.35]","[1758979800000, 202.48, 202.78, 202.43, 202.78]","[1758981600000, 202.79, 203.12, 202.79, 202.98]","[1758983400000, 202.91, 203.34, 202.85, 203.22]"


In [5]:
rows = []

for idx, coin in enumerate(coins):
    total_records = len(payloads[idx])

    for i in range(total_records):
        timestamp_ms = payloads[idx][i][0]
        timestamp = datetime.fromtimestamp(timestamp_ms / 1000)

        open = round(payloads[idx][i][1], 5)
        high = round(payloads[idx][i][2], 5)
        low = round(payloads[idx][i][3], 5)
        close = round(payloads[idx][i][4], 5)

        row = {
            'coin_id': coin,
            'timestamp': timestamp,
            'open_price': open,
            'high_price': high,
            'low_price': low,
            'close_price': close
        }
        rows.append(row)

ohlc_df = pd.DataFrame(rows)
ohlc_df


,coin_id,timestamp,open_price,high_price,low_price,close_price
0,bitcoin,2025-09-26 11:00:00,109057.00,109057.00,108971.00,108971.00
1,bitcoin,2025-09-26 11:30:00,109100.00,109166.00,109020.00,109166.00
2,bitcoin,2025-09-26 12:00:00,109325.00,109380.00,109156.00,109156.00
3,bitcoin,2025-09-26 12:30:00,109099.00,109561.00,109099.00,109546.00
4,bitcoin,2025-09-26 13:00:00,109540.00,109540.00,109385.00,109426.00
...,...,...,...,...,...,...
91,solana,2025-09-27 08:30:00,201.69,201.69,201.22,201.34
92,solana,2025-09-27 09:00:00,201.32,202.44,201.32,202.35
93,solana,2025-09-27 09:30:00,202.48,202.78,202.43,202.78
94,solana,2025-09-27 10:00:00,202.79,203.12,202.79,202.98


# LOAD

In [133]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [134]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")

[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [135]:
sql_table = "coin_ohlc"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")

[SUCCESS] Table 'coin_ohlc' exists in database 'cryptodb'


In [136]:
INSERT_SQL = f"""
INSERT INTO {sql_table} (
    coin_id, timestamp, open_price, high_price, low_price, close_price
) VALUES (
    %s, %s, %s, %s, %s, %s
)
"""

In [137]:
ohlc_list = ohlc_df.values.tolist()

In [138]:
try:
    db_cur.executemany(INSERT_SQL, ohlc_list)
    db_conn.commit()
    print(f"[SUCCESS] Inserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to insert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Inserted 360 records into table 'coin_ohlc'
[INFO] MySQL connection closed
